In [8]:
from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization as BN, GaussianNoise as GN,  Dense, Activation, Flatten, Dropout as DP
from tensorflow.keras.optimizers import SGD, Adamax, Adam
from keras.utils import np_utils

from keras.callbacks import LearningRateScheduler as LRS, ReduceLROnPlateau as RLROP
from keras.preprocessing.image import ImageDataGenerator

In [9]:
batch_size = 64
num_classes = 10
epochs = 300
dropout = 0.2

In [10]:
#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [11]:
## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20,
    zoom_range=[1.0,1.2],
    shear_range = 15,
    horizontal_flip=True)

In [17]:
## DEF NN TOPOLOGY  
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(BN())
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(DP(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(DP(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(DP(0.5))

model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(DP(0.5))

model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(DP(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BN())
model.add(DP(0.5))
model.add(Dense(num_classes, activation='softmax'))    # num_classes = 10

model.summary()


## OPTIM AND COMPILE
opt = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
set_lr = RLROP(factor=0.5, min_lr=1e-6, patience=5)


## TRAINING with DA and LRA
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr],
                            verbose=1)


## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_78 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_85 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_79 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_86 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_80 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_87 (Bat  (None, 32, 32, 32)      

KeyboardInterrupt: 